In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[13]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.0,0.013267,-8.272,0.0,0.0
1,2724.2,0.166936,-8.114,0.0,0.0
2,2622.2,0.161182,-7.898,0.0,0.0
3,2614.2,0.160731,-7.670,0.0,0.0
4,2622.2,0.161182,-7.399,0.0,0.0
...,...,...,...,...,...
43819,3079.5,0.186978,-0.766,0.0,0.0
43820,2985.5,0.181675,-0.742,0.0,0.0
43821,2964.5,0.180491,-0.764,0.0,0.0
43822,2979.5,0.181337,-0.867,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[13]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_SK,Temp_SK
0,0.0,-8.272
1,2724.2,-8.114
2,2622.2,-7.898
3,2614.2,-7.670
4,2622.2,-7.399
...,...,...
43819,3079.5,-0.766
43820,2985.5,-0.742
43821,2964.5,-0.764
43822,2979.5,-0.867


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_SK,Temp_SK,Lag-2,Lag-7
0,3162.6,-9.636,2851.1,0.0
1,3123.6,-9.380,2773.1,2724.2
2,3112.6,-9.129,2755.1,2622.2
3,3181.6,-9.025,2799.1,2614.2
4,3352.6,-9.007,2837.1,2622.2
...,...,...,...,...
43651,3079.5,-0.766,3409.5,2819.5
43652,2985.5,-0.742,3266.5,2796.5
43653,2964.5,-0.764,3117.5,2729.5
43654,2979.5,-0.867,2962.5,2640.5


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 48s 32ms/step - loss: 7.2535e-04 - mape: 5279.0557 - mae: 0.0201 - val_loss: 4.2131e-04 - val_mape: 8.9071 - val_mae: 0.0178
Epoch 2/100
1105/1105 [==============================] - 34s 31ms/step - loss: 4.8291e-04 - mape: 5363.7690 - mae: 0.0171 - val_loss: 3.3507e-04 - val_mape: 8.0611 - val_mae: 0.0150
Epoch 3/100
1105/1105 [==============================] - 35s 32ms/step - loss: 3.6266e-04 - mape: 5378.4600 - mae: 0.0142 - val_loss: 2.7860e-04 - val_mape: 7.0619 - val_mae: 0.0133
Epoch 4/100
1105/1105 [==============================] - 34s 31ms/step - loss: 3.0699e-04 - mape: 5250.9604 - mae: 0.0127 - val_loss: 2.2062e-04 - val_mape: 6.0275 - val_mae: 0.0113
Epoch 5/100
1105/1105 [==============================] - 34s 31ms/step - loss: 2.6828e-04 - mape: 5294.5991 - mae: 0.0116 - val_loss: 8.4721e-04 - val_mape: 12.1703 - val_mae: 0.0254
Epoch 6/100
1105/1105 [==============================] - 34s 31ms/step - loss: 2.2633e-04

Epoch 46/100
1105/1105 [==============================] - 32s 29ms/step - loss: 1.4148e-04 - mape: 4515.2495 - mae: 0.0071 - val_loss: 9.7217e-04 - val_mape: 13.9179 - val_mae: 0.0280
Epoch 47/100
1105/1105 [==============================] - 31s 28ms/step - loss: 1.4245e-04 - mape: 4695.3071 - mae: 0.0071 - val_loss: 4.6964e-04 - val_mape: 9.2213 - val_mae: 0.0189
Epoch 48/100
1105/1105 [==============================] - 34s 31ms/step - loss: 1.3999e-04 - mape: 4562.1279 - mae: 0.0070 - val_loss: 8.0449e-04 - val_mape: 12.4679 - val_mae: 0.0253
Epoch 49/100
1105/1105 [==============================] - 34s 31ms/step - loss: 1.4387e-04 - mape: 4611.0205 - mae: 0.0071 - val_loss: 8.3128e-04 - val_mape: 12.9426 - val_mae: 0.0261
Epoch 50/100
1105/1105 [==============================] - 34s 31ms/step - loss: 1.3935e-04 - mape: 4601.6094 - mae: 0.0070 - val_loss: 9.1506e-04 - val_mape: 13.2248 - val_mae: 0.0262
Epoch 51/100
1105/1105 [==============================] - 34s 31ms/step - loss: 1

1105/1105 [==============================] - 28s 26ms/step - loss: 1.3045e-04 - mape: 4212.3755 - mae: 0.0067 - val_loss: 5.2945e-04 - val_mape: 9.6574 - val_mae: 0.0194
Epoch 91/100
1105/1105 [==============================] - 28s 26ms/step - loss: 1.3278e-04 - mape: 4126.3179 - mae: 0.0068 - val_loss: 5.5963e-04 - val_mape: 10.4781 - val_mae: 0.0210
Epoch 92/100
1105/1105 [==============================] - 29s 26ms/step - loss: 1.3885e-04 - mape: 4277.9204 - mae: 0.0070 - val_loss: 6.6623e-04 - val_mape: 11.3183 - val_mae: 0.0229
Epoch 93/100
1105/1105 [==============================] - 28s 26ms/step - loss: 1.2792e-04 - mape: 4145.5107 - mae: 0.0067 - val_loss: 0.0011 - val_mape: 13.0900 - val_mae: 0.0266
Epoch 94/100
1105/1105 [==============================] - 29s 26ms/step - loss: 1.3160e-04 - mape: 4371.3560 - mae: 0.0068 - val_loss: 4.5442e-04 - val_mape: 9.3432 - val_mae: 0.0187
Epoch 95/100
1105/1105 [==============================] - 29s 26ms/step - loss: 1.3179e-04 - mape: 

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 3329.0832619667053


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 17ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.14970444, 0.14135376, 0.14106674, ..., 0.18235412, 0.18471776,
       0.1864486 ], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.16652941, 0.16636018, 0.16754476, ..., 0.18049053, 0.18133666,
       0.17197283])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,2717.0,2418.730469
1,2714.0,2270.691162
2,2735.0,2265.603027
3,2824.8,2466.028076
4,3117.9,2740.957275
...,...,...
4363,3079.5,2900.448730
4364,2985.5,2950.152100
4365,2964.5,2997.537598
4366,2979.5,3039.439453


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

9.627646250502558